# Import Packages

In [1]:
!pip install sparqlwrapper

In [2]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
def select(query, service='https://query.wikidata.org/sparql'):
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query().convert()
    return pd.json_normalize(result['results']['bindings'])

## 1.1 Olympians

In [ ]:
query_string = """
SELECT
  ?person ?personLabel ?sexgenderLabel
WHERE
{
  ?person wdt:P3171 ?value .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
LIMIT 30000
"""
result=select(query_string)
result.to_csv("Olympians.csv")
result

## 1.2 Olympians with Sex or Gender

In [ ]:
query_string = """
SELECT
  ?person ?personLabel ?sexgenderLabel
WHERE
{
  ?person wdt:P3171 ?value .
  OPTIONAL { ?person wdt:P21 ?sexgender . }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
LIMIT 30000
"""
result=select(query_string)
result.to_csv("Olympians_sex_or_gender.csv")
result

## 1.3 Olympians with Citizenship

In [5]:
query_string = """
SELECT
  ?person (SAMPLE(?personLabel) AS ?personLabel) (GROUP_CONCAT(?citizenshipLabel;SEPARATOR=', ') AS ?citizenship)
WHERE {
  ?person wdt:P3171 [] .
  OPTIONAL { ?person wdt:P27 ?value2 . }
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
    ?person rdfs:label ?personLabel.
    ?value2 rdfs:label ?citizenshipLabel.
  }
}
GROUP BY ?person
"""
result=select(query_string)
result.to_csv("Olympians_citizenship.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,citizenship.type,citizenship.value
0,uri,http://www.wikidata.org/entity/Q221185,en,literal,Kateřina Emmons,literal,Czech Republic
1,uri,http://www.wikidata.org/entity/Q222350,en,literal,Piotr Haczek,literal,Poland
2,uri,http://www.wikidata.org/entity/Q225913,en,literal,Miruts Yifter,literal,Ethiopia
3,uri,http://www.wikidata.org/entity/Q225985,en,literal,Eder Sánchez Arraiz,literal,Mexico
4,uri,http://www.wikidata.org/entity/Q227338,en,literal,Zsolt Borkai,literal,Hungary
...,...,...,...,...,...,...,...
29646,uri,http://www.wikidata.org/entity/Q1400636,en,literal,Gérard Fenouil,literal,France
29647,uri,http://www.wikidata.org/entity/Q1410818,en,literal,Pierre Pujol,literal,France
29648,uri,http://www.wikidata.org/entity/Q1420688,en,literal,Maria-Joëlle Conjungo,literal,Central African Republic
29649,uri,http://www.wikidata.org/entity/Q1430388,en,literal,Zelimkhan Huseynov,literal,"Russia, Azerbaijan, Soviet Union"


## 1.4 Olympians with Sports

In [ ]:
query_string = """
SELECT
  ?person ?personLabel (GROUP_CONCAT(?sportLabel;SEPARATOR=', ') AS ?sports)
WHERE
{
  ?person wdt:P3171 ?value .
  OPTIONAL { ?person wdt:P106 ?sport .
    ?sport rdfs:label ?sportLabel . FILTER(LANG(?sportLabel)='en') }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel
LIMIT 30000
"""
result=select(query_string)
result.to_csv("Olympians_sports.csv")
result

## 1.5 Olympians with All Demographic Categories ***except citizenship***

In [ ]:
query_string = """
SELECT
  ?person ?personLabel ?sexgenderLabel (GROUP_CONCAT(?sportLabel;SEPARATOR=', ') AS ?sports)
WHERE
{
  ?person wdt:P3171 ?value .
  OPTIONAL { ?person wdt:P21 ?sexgender . }
  OPTIONAL { ?person wdt:P106 ?sport .
           ?sport rdfs:label ?sportLabel . FILTER(LANG(?sportLabel)='en') }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel ?sexgenderLabel
LIMIT 30000
"""
result=select(query_string)
result.to_csv("Olympians_complete.csv")
result

## 2.1 People With Medical Conditions

In [ ]:
query_string = """
SELECT ?person ?personLabel (GROUP_CONCAT(DISTINCT ?conditionLabel;SEPARATOR=', ') AS ?conditions) 
WHERE
{
  ?person wdt:P31 wd:Q5 .
  ?person wdt:P1050 ?condition .
  ?condition rdfs:label ?conditionLabel . FILTER(LANG(?conditionLabel)='en')
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel
"""

result=select(query_string)
result.to_csv("Medical_conditions.csv")
result

## 2.2 People With Medical Conditions and Sex or Gender

In [ ]:
query_string = """
SELECT 
  ?person ?personLabel ?sexgenderLabel (GROUP_CONCAT(DISTINCT ?conditionLabel;SEPARATOR=', ') AS ?conditions) 
WHERE
{
  ?person wdt:P31 wd:Q5 .
  ?person wdt:P1050 ?condition .
  ?condition rdfs:label ?conditionLabel . FILTER(LANG(?conditionLabel)='en')
  OPTIONAL { ?person wdt:P21 ?sexgender . }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel ?sexgenderLabel
"""

result=select(query_string)
result.to_csv("Medical_conditions_sex_or_gender.csv")
result

## 2.3 People With Medical Conditions and Citizenship

In [ ]:
query_string = """
SELECT 
  ?person ?personLabel (GROUP_CONCAT(DISTINCT ?citizenshipLabel;SEPARATOR=', ') AS ?citizenship) 
  (GROUP_CONCAT(DISTINCT ?conditionLabel;SEPARATOR=', ') AS ?conditions) 
WHERE
{
  ?person wdt:P31 wd:Q5 .
  ?person wdt:P1050 ?condition .
  ?condition rdfs:label ?conditionLabel . FILTER(LANG(?conditionLabel)='en')
  OPTIONAL { ?person wdt:P27 ?value . 
           ?value rdfs:label ?citizenshipLabel . FILTER (LANG(?citizenshipLabel)='en') }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel
"""

result=select(query_string)
result.to_csv("Medical_conditions_citizenship.csv")
result

## 2.4 People With Medical Conditions and Cause of Death

In [ ]:
query_string = """
SELECT ?person ?personLabel (GROUP_CONCAT(DISTINCT ?conditionLabel;SEPARATOR=', ') AS ?conditions) (GROUP_CONCAT(DISTINCT ?causedeathLabel;SEPARATOR=', ') AS ?causedeath)
WHERE
{
  ?person wdt:P31 wd:Q5 .
  ?person wdt:P1050 ?condition .
  ?condition rdfs:label ?conditionLabel . FILTER(LANG(?conditionLabel)='en')
  OPTIONAL { ?person wdt:P509 ?causedeath .
           ?causedeath rdfs:label ?causedeathLabel . FILTER(LANG(?causedeathLabel)='en') }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel
"""

result=select(query_string)
result.to_csv("Medical_conditions_cause_of_death.csv")
result

## 2.5 People With Sexually-Transmitted Infections

In [ ]:
query_string = """
SELECT ?person ?personLabel (GROUP_CONCAT(DISTINCT ?conditionLabel;SEPARATOR=', ') AS ?conditions) 
WHERE
{
  ?person wdt:P31 wd:Q5 .
  ?person wdt:P1050 ?condition .
  ?condition wdt:P279 wd:Q12198 .
  ?condition rdfs:label ?conditionLabel . FILTER(LANG(?conditionLabel)='en')
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel
"""

result=select(query_string)
result.to_csv("Medical_conditions_STIs.csv")
result

## 2.6 People With STIs, Sex or Gender, Citizenship, and Cause of Death

In [4]:
query_string = """
SELECT ?person ?personLabel ?sexgenderLabel (GROUP_CONCAT(DISTINCT ?citizenshipLabel;SEPARATOR=', ') AS ?citizenship) 
  (GROUP_CONCAT(DISTINCT ?conditionLabel;SEPARATOR=', ') AS ?conditions) 
  (GROUP_CONCAT(DISTINCT ?causedeathLabel;SEPARATOR=', ') AS ?causedeath)
WHERE
{
  ?person wdt:P31 wd:Q5 .
  OPTIONAL { ?person wdt:P21 ?sexgender . }
  ?person wdt:P1050 ?condition .
  ?condition wdt:P279+ wd:Q12198 .
  ?condition rdfs:label ?conditionLabel . FILTER(LANG(?conditionLabel)='en')
  OPTIONAL { ?person wdt:P27 ?value . 
           ?value rdfs:label ?citizenshipLabel . FILTER (LANG(?citizenshipLabel)='en') }
  OPTIONAL { ?person wdt:P509 ?causedeath .
           ?causedeath rdfs:label ?causedeathLabel . FILTER(LANG(?causedeathLabel)='en') }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel ?sexgenderLabel
ORDER BY ?personLabel
"""

result=select(query_string)
result.to_csv("Medical_conditions_STIs_complete.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,sexgenderLabel.xml:lang,sexgenderLabel.type,sexgenderLabel.value,citizenship.type,citizenship.value,conditions.type,conditions.value,causedeath.type,causedeath.value
0,uri,http://www.wikidata.org/entity/Q831387,en,literal,Abing,en,literal,male,literal,People's Republic of China,literal,syphilis,literal,
1,uri,http://www.wikidata.org/entity/Q58640157,en,literal,Adam Johnson,en,literal,male,literal,,literal,HIV/AIDS,literal,death from AIDS-related complications
2,uri,http://www.wikidata.org/entity/Q352,en,literal,Adolf Hitler,en,literal,male,literal,"Cisleithania, First Republic of Austria, Repub...",literal,syphilis,literal,ballistic trauma
3,uri,http://www.wikidata.org/entity/Q2740907,en,literal,Agustín Gómez-Arcos,en,literal,male,literal,"France, Spain",literal,HIV/AIDS,literal,death from AIDS-related complications
4,uri,http://www.wikidata.org/entity/Q80048,en,literal,Al Capone,en,literal,male,literal,United States of America,literal,"neurosyphilis, gonorrhea, syphilis",literal,bronchopneumonia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,uri,http://www.wikidata.org/entity/Q17627823,en,literal,Víctor Hugo Pérez Peñaloza,en,literal,genderfluid,literal,Chile,literal,HIV/AIDS,literal,encephalitis
150,uri,http://www.wikidata.org/entity/Q109630123,en,literal,William Way,en,literal,male,literal,United States of America,literal,HIV/AIDS,literal,
151,uri,http://www.wikidata.org/entity/Q3570682,en,literal,Xavier Fourcade,en,literal,male,literal,United States of America,literal,HIV/AIDS,literal,death from AIDS-related complications
152,uri,http://www.wikidata.org/entity/Q3573333,en,literal,Yvan Burger,en,literal,male,literal,France,literal,HIV/AIDS,literal,death from AIDS-related complications


## 3.1 University of Virginia Employees

In [ ]:
query_string = """
SELECT
  ?person ?personLabel
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
}
"""
result=select(query_string)
result.to_csv("UVA_employees.csv")
result

## 3.2 UVA Employees with Sex or Gender

In [ ]:
query_string = """
SELECT
  ?person ?personLabel ?sexorgender
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P21 ?value .
    ?value rdfs:label ?sexorgender . FILTER(LANG(?sexorgender)='en') }
}
"""
result=select(query_string)
result.to_csv("UVA_employees_sex_or_gender.csv")
result

## 3.3 UVA Employees with Ethnicity

In [ ]:
query_string = """
SELECT
  ?person ?personLabel (GROUP_CONCAT(DISTINCT ?ethnicityLabel;SEPARATOR=', ') AS ?ethnicity)
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P172 ?ethnicity .
    ?ethnicity rdfs:label ?ethnicityLabel . FILTER(LANG(?ethnicityLabel)='en') }
}
GROUP BY ?person ?personLabel
"""
result=select(query_string)
result.to_csv("UVA_employees_ethnicity.csv")
result

## 3.4 UVA Employees with Citizenship

In [ ]:
query_string = """
SELECT
  ?person ?personLabel (GROUP_CONCAT(DISTINCT ?citizenshipLabel;SEPARATOR=', ') AS ?citizenship)
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P27 ?citizenship .
    ?citizenship rdfs:label ?citizenshipLabel . FILTER(LANG(?citizenshipLabel)='en') }
}
GROUP BY ?person ?personLabel
"""
result=select(query_string)
result.to_csv("UVA_employees_citizenship.csv")
result

## 3.5 UVA Employees with Religion

In [ ]:
query_string = """
SELECT
  ?person ?personLabel ?religion
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P140 ?value .
    ?value rdfs:label ?religion . FILTER(LANG(?religion)='en') }
}
"""
result=select(query_string)
result.to_csv("UVA_employees_religion.csv")
result

## 3.6 UVA Employees with Sexuality

In [ ]:
query_string = """
SELECT
  ?person ?personLabel ?sexuality
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P91 ?value4 .
    ?value4 rdfs:label ?sexuality . FILTER(LANG(?sexuality)='en') }
}
"""
result=select(query_string)
result.to_csv("UVA_employees_sexuality.csv")
result

## 3.7 UVA Employees with Software Development

In [ ]:
query string = """
SELECT
  ?person ?personLabel ?developed
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P178+ ?value .
    ?value rdfs:label ?developed . FILTER(LANG(?developed)='en') }
}
"""
result=select(query_string)
result.to_csv("UVA_employees_development.csv")
result

## 3.8 UVA Employees with All Categories

In [ ]:
query_string = """
SELECT
  ?person ?personLabel ?sexorgender (GROUP_CONCAT(DISTINCT ?ethnicityLabel;SEPARATOR=', ') AS ?ethnicity) 
  (GROUP_CONCAT(DISTINCT ?citizenshipLabel;SEPARATOR=', ') AS ?citizenship) ?religion 
  ?sexuality ?developed
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P21 ?value .
           ?value rdfs:label ?sexorgender . FILTER(LANG(?sexorgender)='en') }
  OPTIONAL { ?person wdt:P172 ?ethnicity .
           ?ethnicity rdfs:label ?ethnicityLabel . FILTER(LANG(?ethnicityLabel)='en') }
  OPTIONAL { ?person wdt:P27 ?citizenship .
           ?citizenship rdfs:label ?citizenshipLabel . FILTER(LANG(?citizenshipLabel)='en') }
  OPTIONAL { ?person wdt:P140 ?value2 .
           ?value2 rdfs:label ?religion . FILTER(LANG(?religion)='en') }
  OPTIONAL { ?person wdt:P91 ?value3 .
           ?value3 rdfs:label ?sexuality . FILTER(LANG(?sexuality)='en') }
  OPTIONAL { ?person wdt:P178+ ?value4 .
           ?value4 rdfs:label ?developed . FILTER(LANG(?developed)='en') }
}
GROUP BY ?person ?personLabel ?sexorgender ?religion ?sexuality ?developed
"""

result=select(query_string)
result.to_csv("UVA_employees_complete.csv")
result